In [1]:
import pandas as pd
import os
import numpy as np
import epiweeks
import datetime
import requests
import zipfile
import csv
from contextlib import ExitStack
import progressbar as pbar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./data/Data/Poland_deaths_2020.csv', sep = ';')
data = data.iloc[:-3, :]
data = data.loc[:, ['Data', 'Wiek', 'Zgony (ogółem)']]

data.loc[:, 'Zgony (ogółem)'] = data.loc[:, 'Zgony (ogółem)'].apply(pd.to_numeric)
data.loc[:, 'Data'] = data.loc[:, 'Data'].apply(pd.to_datetime)
data = data.groupby(['Data', 'Wiek']).sum().reset_index()
result = pd.DataFrame(columns=['Data', 'location', 'location_name', 'Age group', 'Zgony (ogółem)'])

result = result.append(data[data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 24].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('00-24')

result = result.append(data[np.logical_and(data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 34, data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) > 24)].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('25-34')

result = result.append(data[np.logical_and(data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 44, data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) > 34)].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('35-44')

result = result.append(data[np.logical_and(data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 54, data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) > 44)].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('45-54')

result = result.append(data[np.logical_and(data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 64, data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) > 54)].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('55-64')

result = result.append(data[np.logical_and(data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 74, data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) > 64)].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('65-74')

result = result.append(data[np.logical_and(data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) <= 84, data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) > 74)].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('75-84')

result = result.append(data[data['Wiek'].apply(lambda x: int(x) if x not in ['90+', 'b.d.'] else 90) >= 85].groupby(['Data']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('85+')



result = result.rename(columns={'Data': 'date', 'Zgony (ogółem)': 'value'})
result['location'] = result['location'].fillna('PL')
result['location_name'] = result['location_name'].fillna('Poland')
result1 = result


data = pd.read_csv('./data/Data/Poland_deaths_2021_2022.csv', encoding='latin-1', sep = ';')
data = data.loc[:, ['data_rap_zgonu', 'kat_wiek', 'liczba_zaraportowanych_zgonow']]
data = data.groupby(['data_rap_zgonu', 'kat_wiek']).sum().reset_index()

data.loc[:, 'liczba_zaraportowanych_zgonow'] = data.loc[:, 'liczba_zaraportowanych_zgonow'].apply(pd.to_numeric)
data.loc[:, 'data_rap_zgonu'] = data.loc[:, 'data_rap_zgonu'].apply(pd.to_datetime)
result = pd.DataFrame(columns=['data_rap_zgonu', 'location', 'location_name', 'Age group', 'liczba_zaraportowanych_zgonow'])

result = result.append(data[data['kat_wiek'].isin(['0-18', '19-24'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('00-24')

result = result.append(data[data['kat_wiek'].isin(['25-34'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('25-34')

result = result.append(data[data['kat_wiek'].isin(['35-44'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('35-44')

result = result.append(data[data['kat_wiek'].isin(['45-54'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('45-54')

result = result.append(data[data['kat_wiek'].isin(['55-64'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('55-64')

result = result.append(data[data['kat_wiek'].isin(['65-74'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('65-74')

result = result.append(data[data['kat_wiek'].isin(['75-84'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('75-84')

result = result.append(data[data['kat_wiek'].isin(['85-94', '95+', 'BD'])].groupby(['data_rap_zgonu']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('85+')

result = result.rename(columns={'data_rap_zgonu': 'date', 'liczba_zaraportowanych_zgonow': 'value'})
result['location'] = result['location'].fillna('PL')
result['location_name'] = result['location_name'].fillna('Poland')
result = result.append(result1)
result['date'] = result['date'].apply(pd.to_datetime)
result = result.sort_values('date')

for age_group in result['Age group'].unique():
    result.loc[result['Age group'] == age_group, 'value'] = result[result['Age group'] == age_group]['value'].cumsum()
fixed_deaths = pd.DataFrame(columns=['date', 'location', 'location_name', 'Age group', 'value'])

for age_group in result['Age group'].unique():
        deaths_age_group = result[result['Age group'] == age_group].groupby('date')['value'].sum().reset_index()

        fixed_deaths_age_group = deaths_age_group['value']
        fixed_deaths_age_group.index = deaths_age_group['date']
        fixed_deaths_age_group = fixed_deaths_age_group.resample('D').mean()
        fixed_deaths_age_group = fixed_deaths_age_group.interpolate()
        
        removed_negative_deaths = np.insert(np.maximum(0, (fixed_deaths_age_group.values[1:] - fixed_deaths_age_group.values[:-1])), 0, 0)
        fixed_deaths_age_group = pd.DataFrame(fixed_deaths_age_group)
        fixed_deaths_age_group['value'] = removed_negative_deaths
        fixed_deaths = fixed_deaths.append(fixed_deaths_age_group.reset_index())
        fixed_deaths['location'] = fixed_deaths['location'].fillna('Poland')
        fixed_deaths['location_name'] = fixed_deaths['location_name'].fillna('PL')
        fixed_deaths['Age group'] = fixed_deaths['Age group'].fillna(age_group)
fixed_deaths = fixed_deaths.sort_values('date')

country = 'Poland'
today = datetime.date.today().strftime("%d-%m-%Y")
today = '06-03-2022'
data_dir_path = f'data/World/{today}'
if not os.path.exists(f'{data_dir_path}'):
    os.mkdir(f'{data_dir_path}')
if not os.path.exists(f'{data_dir_path}/{country}'):
    os.mkdir(f'{data_dir_path}/{country}')
    
fixed_deaths.to_csv(f'{data_dir_path}/{country}/covid_deaths.csv')

In [3]:
data1 = pd.read_csv('./data/Data/Poland_cases_2020_part_1.csv', encoding='latin-1', sep = ',')
data1 = data1.drop(columns=['Unnamed: 0', 'A_ID_WOJEWODZTWO'])
data1 = data1.groupby(['Z_DATA_WYNIKU', 'gr_wiek']).sum().reset_index()
data1 = data1.rename(columns={'Z_DATA_WYNIKU': 'date', 'gr_wiek': 'Age group', 'ile': 'value'})
data1['date'] = data1['date'].apply(pd.to_datetime)

data = pd.read_csv('./data/Data/Poland_cases_2020_part_2.csv', encoding='latin-1', sep = ',')
data = data.drop(columns=['Unnamed: 0', 'teryt_woj', 'teryt_pow', 'teryt_gmn'])
data = data.groupby(['data_wpis_pierw_wyn_poz', 'kat_wiek']).sum().reset_index()
data = data.rename(columns={'data_wpis_pierw_wyn_poz': 'date', 'kat_wiek': 'Age group', 'liczba_nowych_osob_zakazonych': 'value'})
data['date'] = data['date'].apply(pd.to_datetime)
data['Age group'] = data['Age group'].apply(lambda x: x[:-4])
data = data.append(data1)
data = data[data['Age group'] != 'b.d.'].sort_values('date')

population = pd.read_csv(f'{data_dir_path}/{country}/old_population.csv')
population = population[~population['Age group'].isin(['UNK', 'Y_OPEN'])]
population['Age group'] = population['Age group'].replace({'TOTAL': 'YTOTAL', 'Y_LT1': 'Y0'}).apply(lambda x: x[1:])
population_yearly = population[~population['Age group'].isin(['TOTAL', '0'])]
for start, stop in zip(range(0, 96, 5), range(5, 101, 5)): 
    population = population.append(population[population['Age group'].isin([str(i) for i in range(start, stop)])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
    population = population[~population['Age group'].isin([str(i) for i in range(start, stop)])]
    population['Age group'] = population['Age group'].fillna(f'{start:02d}-{stop-1:02d}')
population = population.append(population[population['Age group'].isin(['80-84', '85-89', '90-94', '95-99'])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
population = population[~population['Age group'].isin(['85-89', '90-94', '95-99'])]
population['Age group'] = population['Age group'].fillna(f'85+')
population = population[population['Year'] == 2020]
population = population[population['Sex'] == 'T']
population = population.drop(columns=['Year', 'Region', 'Sex'])
population = population.set_index('Age group')

new_data = pd.DataFrame(columns=['date', 'Age group'])
x = data.set_index('date')
new_data = new_data.append((x[x['Age group'] == '0-10']['value'] + x[x['Age group'] == '11-20']['value'] + 
                            x[x['Age group'] == '21-30']['value'].apply(lambda x: x * population.loc['20-24'] / (population.loc['20-24'] + population.loc['25-29']))['Value']).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'00-24')
new_data = new_data.rename(columns={0: 'Value'})
new_data = new_data.append((x[x['Age group'] == '21-30']['value'].apply(lambda x: x * population.loc['25-29'] / (population.loc['20-24'] + population.loc['25-29'])) +
                            x[x['Age group'] == '31-40']['value'].apply(lambda x: x * population.loc['30-34'] / (population.loc['30-34'] + population.loc['35-39']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'25-35')

new_data = new_data.append((x[x['Age group'] == '41-50']['value'].apply(lambda x: x * population.loc['40-44'] / (population.loc['40-44'] + population.loc['45-49'])) +
                            x[x['Age group'] == '31-40']['value'].apply(lambda x: x * population.loc['35-39'] / (population.loc['30-34'] + population.loc['35-39']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'35-44')

new_data = new_data.append((x[x['Age group'] == '41-50']['value'].apply(lambda x: x * population.loc['45-49'] / (population.loc['40-44'] + population.loc['45-49'])) +
                            x[x['Age group'] == '51-60']['value'].apply(lambda x: x * population.loc['50-54'] / (population.loc['50-54'] + population.loc['55-59']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'45-54')

new_data = new_data.append((x[x['Age group'] == '61-70']['value'].apply(lambda x: x * population.loc['60-64'] / (population.loc['60-64'] + population.loc['65-69'])) +
                            x[x['Age group'] == '51-60']['value'].apply(lambda x: x * population.loc['55-59'] / (population.loc['50-54'] + population.loc['55-59']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'55-64')

new_data = new_data.append((x[x['Age group'] == '61-70']['value'].apply(lambda x: x * population.loc['65-69'] / (population.loc['60-64'] + population.loc['65-69'])) +
                            x[x['Age group'] == '71-80']['value'].apply(lambda x: x * population.loc['70-74'] / (population.loc['70-74'] + population.loc['75-79']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'65-74')

new_data = new_data.append((x[x['Age group'] == '81-90']['value'].apply(lambda x: x * population.loc['80-84'] / (population.loc['80-84'] + population.loc['85+'])) +
                            x[x['Age group'] == '71-80']['value'].apply(lambda x: x * population.loc['75-79'] / (population.loc['70-74'] + population.loc['75-79']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'75-84')

new_data = new_data.append((x[x['Age group'] == '81-90']['value'].apply(lambda x: x * population.loc['80-84'] / (population.loc['80-84'] + population.loc['85+'])) +
                            x[x['Age group'] == '71-80']['value'].apply(lambda x: x * population.loc['75-79'] / (population.loc['70-74'] + population.loc['75-79']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'75-84')

new_data = new_data.append((x[x['Age group'] == '81-90']['value'].apply(lambda x: x * population.loc['85+'] / (population.loc['80-84'] + population.loc['85+']))).reset_index())
new_data['Age group'] = new_data['Age group'].fillna(f'85+')

new_data = new_data.fillna(0.0).rename(columns={'Value': 'value'})

data = pd.read_csv('./data/Data/Poland_cases_2021_2022.csv', encoding='latin-1', sep = ';')
data = data.drop(columns=['teryt_woj', 'teryt_pow', 'wiek', 'producent', 'dawka_ost', 'numer_zarazenia'])
data = data.groupby(['data_rap_zakazenia', 'kat_wiek']).sum().reset_index()

result = pd.DataFrame(columns=['data_rap_zakazenia', 'location', 'location_name', 'Age group', 'liczba_zaraportowanych_zakazonych'])

result = result.append(data[data['kat_wiek'].isin(['0-18', '19-24'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('00-24')

result = result.append(data[data['kat_wiek'].isin(['25-34'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('25-34')

result = result.append(data[data['kat_wiek'].isin(['35-44'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('35-44')

result = result.append(data[data['kat_wiek'].isin(['45-54'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('45-54')

result = result.append(data[data['kat_wiek'].isin(['55-64'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('55-64')

result = result.append(data[data['kat_wiek'].isin(['65-74'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('65-74')

result = result.append(data[data['kat_wiek'].isin(['75-84'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('75-84')

result = result.append(data[data['kat_wiek'].isin(['85-94', '95+', 'BD'])].groupby(['data_rap_zakazenia']).sum().reset_index())
result['Age group'] = result['Age group'].fillna('85+')

result = result.rename(columns={'data_rap_zakazenia': 'date', 'liczba_zaraportowanych_zakazonych': 'value'})

new_data = result.append(new_data[new_data['date'] < datetime.datetime.strptime('2021-01-02', '%Y-%m-%d')])
new_data['date'] = new_data['date'].apply(pd.to_datetime)
new_data = new_data.sort_values('date').reset_index()
new_data['location'] = new_data['location'].fillna('PL')
new_data['location_name'] = new_data['location_name'].fillna('Poland')
new_data

for age_group in new_data['Age group'].unique():
    new_data.loc[new_data['Age group'] == age_group, 'value'] = new_data[new_data['Age group'] == age_group]['value'].cumsum()

    
for age_group in new_data['Age group'].unique():
    deaths_age_group = new_data[new_data['Age group'] == age_group].groupby('date')['value'].sum().reset_index()

    fixed_deaths_age_group = deaths_age_group['value']
    fixed_deaths_age_group.index = deaths_age_group['date']
    fixed_deaths_age_group = fixed_deaths_age_group.resample('D').mean()
    fixed_deaths_age_group = fixed_deaths_age_group.interpolate()
        
    removed_negative_deaths = np.insert(np.maximum(0, (fixed_deaths_age_group.values[1:] - fixed_deaths_age_group.values[:-1])), 0, 0)
    fixed_deaths_age_group = pd.DataFrame(fixed_deaths_age_group)
    fixed_deaths_age_group['value'] = removed_negative_deaths
    fixed_deaths = fixed_deaths.append(fixed_deaths_age_group.reset_index())
    fixed_deaths['location'] = fixed_deaths['location'].fillna('Poland')
    fixed_deaths['location_name'] = fixed_deaths['location_name'].fillna('PL')
    fixed_deaths['Age group'] = fixed_deaths['Age group'].fillna(age_group)
fixed_deaths = fixed_deaths.sort_values('date').reset_index(drop=True).drop_duplicates(['date', 'Age group'])
fixed_deaths
country = 'Poland'
today = datetime.date.today().strftime("%d-%m-%Y")
today = '06-03-2022'
data_dir_path = f'data/World/{today}'
if not os.path.exists(f'{data_dir_path}'):
    os.mkdir(f'{data_dir_path}')
if not os.path.exists(f'{data_dir_path}/{country}'):
    os.mkdir(f'{data_dir_path}/{country}')
    
fixed_deaths.to_csv(f'{data_dir_path}/{country}/covid_cases.csv')